# Homework

1. Try to improve cross_val_score result using tf*idf, n-gramms, other classifiers, word2vec (see gensim word2vec tutorial) or something else
2. Write in the same ipython notebook (in Markdown cells) the problem formulation for sentiment a

## Task - try to improve results of classification
### Best result that we had: 
1. [TfidfVectorizer(), TruncatedSVD(n_components=1000), LinearSVC()] 
cross_val_score: **0.841010171848**
2. [CountVectorizer(), TfidfTransformer(), LinearSVC()] 
cross_val_score: **0.845507183831**

In [3]:
from nltk.corpus import movie_reviews
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

print(negids[:5])

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


In [4]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [6]:
pipeline_SVC = Pipeline([
            ("vectorizer", CountVectorizer(ngram_range=(1, 2))),
            ("transformer", TfidfTransformer()),
            ("classifier", LinearSVC())
            ])

### Using *CountVectorizer(ngram_range=(1, 2))*:

In [7]:
%%time
print(cross_val_score(pipeline_SVC,
                      texts, 
                      labels
                     ).mean())

0.830511649374
CPU times: user 20.6 s, sys: 404 ms, total: 21 s
Wall time: 21 s


### Using *CountVectorizer(ngram_range=(1, 5))*:

In [8]:
%%time
print(cross_val_score(pipeline_SVC,
                      texts, 
                      labels
                     ).mean())

0.830511649374
CPU times: user 19.7 s, sys: 288 ms, total: 20 s
Wall time: 20 s


### Using *MultinomialNB* as classifier:

In [9]:
from sklearn.naive_bayes import MultinomialNB

In [10]:
pipeline_NB = Pipeline([
            ("vectorizer", CountVectorizer(ngram_range=(1, 2))),
            ("transformer", TfidfTransformer()),
            ("classifier", MultinomialNB())
            ])

In [11]:
%%time
print(cross_val_score(pipeline_NB,
                      texts, 
                      labels
                     ).mean())

0.829005652359
CPU times: user 21.4 s, sys: 340 ms, total: 21.7 s
Wall time: 21.7 s


### Using *Word2Vec*:

In [1]:
import gensim

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

### For the whole dataset I don't have enough RAM =(

### Save model with *pickle*:

In [12]:
import pickle

In [13]:
pipeline_NB.fit(texts, labels)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
       ...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [15]:
filename = 'NB_model.pkl'
pickle.dump(pipeline_NB, open(filename, 'wb'))